In [1]:
!pip install folium
import folium
from folium import Marker
from folium import plugins
from folium import GeoJson
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [35]:
# https://www.kaggle.com/datasets/sewonghwang/starbucks-seoul
df = pd.read_csv('../input/starbucks-seoul/Starbucks_Seoul.csv')

# 지역 구분을 위한 json 파일 불러오기
geo="../input/starbucks-seoul/Seoul_Gu.json" 

In [5]:
df.head()

,name,address,gu_name,latitude,longitude
0,GS타워,서울특별시 강남구 논현로 508 (역삼동),강남구,37.501859,127.037278
1,SSG마켓도곡R,"서울특별시 강남구 언주로30길 57, 타워팰리스Ⅱ F 지하1층 (도곡동)",강남구,37.490298,127.054895
2,W-Mall,서울특별시 금천구 디지털로 188 (가산동),금천구,37.477305,126.887691
3,가든파이브,서울특별시 송파구 충민로 10 (문정동) 가든파이브툴,송파구,37.478232,127.119370
4,가락본동,서울특별시 송파구 송파대로30길 13 (가락동),송파구,37.494895,127.118785


### Basic Map

In [14]:
# 기본 지도 시각화 (서울의 위도, 경도 입력)

m = folium.Map(location=[37.541, 126.986], zoom_start=12)
m

In [21]:
# 지도 형태 변경
m = folium.Map(location=[37.541, 126.986], zoom_start=12)

# 원하는 좌표에 반경(radius) 표시 (남산)
folium.CircleMarker([37.5538, 126.9810],radius=50, 
                    popup='Laurelhurst Park', color='#3246cc', 
                    fill_color='#3246cc').add_to(m)

# 원하는 좌표에 포인트 표시 (남산)
folium.Marker([37.5538, 126.9810], popup='The Waterfront').add_to(m)
    
m

### Cluster Marker

In [26]:
# 서울 지도에 스타벅스 지점 수 시각화

m = folium.Map([37.541, 126.986], zoom_start=12 ,width="100%", height="100%")
locations = list(zip(df.latitude, df.longitude))
cluster = plugins.MarkerCluster(locations=locations,                     
               popups=df["name"].tolist())  
m.add_child(cluster)
m

### Dot Map

In [28]:
# 서울 지도에 스타벅스 지점수 도드맵 시각화

m = folium.Map(location=[37.541, 126.986], zoom_start=12, width="100%", height="100%")
locations = list(zip(df.latitude, df.longitude))
for i in range(len(locations)):
    folium.CircleMarker(location=locations[i],radius=1).add_to(m)
m

### Bubble Map

In [29]:
# 서울 구 별 스타벅스 지점 수 집계 및 중심점 산출
df_m = df.groupby('gu_name').agg({'latitude':'mean',
                                  'longitude':'mean',
                                  'name':'count'}).reset_index()
df_m.head()

,gu_name,latitude,longitude,name
0,강남구,37.507603,127.044611,80
1,강동구,37.539914,127.137106,14
2,강북구,37.626866,127.026372,5
3,강서구,37.555716,126.841528,16
4,관악구,37.481759,126.944286,11


In [36]:
# 서울 구 별 스타벅스 지점 수 버블맵 시각화

# 기본 지도 생성
m = folium.Map(location=[37.541, 126.986], tiles='Cartodb Positron', 
               zoom_start=11, width="100%", 
               height="100%")

# 구별 구분선, 색상 설정
folium.Choropleth(
    geo_data=geo, # 앞에서 불러온 json 파일 적용
    fill_color="gray"
    ).add_to(m)

# 버블맵 삽입
locations = list(zip(df_m.latitude, df_m.longitude))
for i in range(len(locations)):
    row = df_m.iloc[i]
    folium.CircleMarker(location=locations[i],
                        radius= float(row.name/2), # 버블 크기 설정
                        fill_color="blue"
                       ).add_to(m)
m

# radius에 카운팅을 넣어줌
# 만약 원이 커서 겹치면 float(row.name/1)의 분모값을 조정

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 1974: invalid start byte